# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` を付けない）
  ※本問題は仕様で **`visit_date` 昇順** を要求 → 最終行に `ORDER BY visit_date`
* `NOT IN` は NULL 罠のため回避
* 判定は **ID 基準**（連続 ID かつ各行 `people >= 100`）、表示は仕様どおりの列名と順序

## 1) 問題

* `3 つ以上の連続した id を持ち、各行の people >= 100 のレコードを表示する。結果は visit_date 昇順。`
* 入力テーブル例: `Stadium(id INT, visit_date DATE, people INT)`
* 出力仕様: `id, visit_date, people` を **連続 ID の島（gaps-and-islands）** のうち長さ ≥ 3 の行のみ。最終並びは `visit_date ASC`。

## 2) 最適解（単一クエリ）

> people ≥ 100 を先に絞り込み、`id - ROW_NUMBER()` で **連続 ID の島キー** を作り、長さ ≥ 3 の島だけ残す。

```sql
WITH pre AS (
  SELECT id, visit_date, people
  FROM Stadium
  WHERE people >= 100
),
grp AS (
  SELECT
    id,
    visit_date,
    people,
    id - ROW_NUMBER() OVER (ORDER BY id) AS grp_key
  FROM pre
),
big_islands AS (
  SELECT grp_key
  FROM grp
  GROUP BY grp_key
  HAVING COUNT(*) >= 3
)
SELECT
  g.id,
  g.visit_date,
  g.people
FROM grp AS g
JOIN big_islands AS b
  USING (grp_key)
ORDER BY g.visit_date;

Runtime 352 ms
Beats 56.94%

```

## 3) 代替解

> `LAG` を使って「連続しているか」をフラグ化し、累積和で島を採番する方法。

```sql
WITH pre AS (
  SELECT id, visit_date, people
  FROM Stadium
  WHERE people >= 100
),
marked AS (
  SELECT
    id,
    visit_date,
    people,
    CASE WHEN id = LAG(id) OVER (ORDER BY id) + 1 THEN 0 ELSE 1 END AS is_break
  FROM pre
),
islands AS (
  SELECT
    id,
    visit_date,
    people,
    SUM(is_break) OVER (ORDER BY id) AS grp_key
  FROM marked
),
big_islands AS (
  SELECT grp_key
  FROM islands
  GROUP BY grp_key
  HAVING COUNT(*) >= 3
)
SELECT i.id, i.visit_date, i.people
FROM islands AS i
JOIN big_islands AS b USING (grp_key)
ORDER BY i.visit_date;

Runtime 335 ms
Beats 76.87%

```

## 4) 要点解説

* **判定基準は ID の連続**：日付は連続でなくてよい（問題文のとおり）。
* **Gaps-and-Islands パターン**：`id - ROW_NUMBER()` が同じ値の集合は ID が連番の「島」になる。
* 先に `people >= 100` を絞ることでウィンドウ行数を縮小し、高速化。
* `NOT IN` 不使用。結合は `JOIN ... USING (grp_key)` を採用。
* 並び順は仕様に従い **`visit_date ASC`**。

## 5) 計算量（概算）

* フィルタ後レコード数を `n` とすると：

  * ウィンドウ関数（`ROW_NUMBER` / `LAG`）: **O(n log n)**（`ORDER BY id`）
  * `GROUP BY grp_key`: **O(n)**〜**O(n log n)**
  * 結合: **O(n)** 近似
* インデックス推奨: `PRIMARY KEY(id)` / `INDEX(people)`（`people >= 100` の選択度が高いほど効く）

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Stadium] --> B[前処理 people >= 100]
  B --> C[ウィンドウ id - ROW_NUMBER で島キー]
  C --> D[島ごとに COUNT>=3 を抽出]
  D --> E[該当島と結合して投影]
  E --> F[visit_date 昇順で出力]
```

まだ少しだけ速く・シンプルにできます。主な改善点は **`big_islands` との結合をやめて、ウィンドウ `COUNT()` で島の長さを直接フィルタ**することと、**適切なインデックス**です。

---

## 改善版（JOIN 排除・1 回のスキャンで判定）

```sql
WITH pre AS (
  SELECT id, visit_date, people
  FROM Stadium
  WHERE people >= 100
),
grp AS (
  SELECT
    id,
    visit_date,
    people,
    id - ROW_NUMBER() OVER (ORDER BY id) AS grp_key
  FROM pre
)
SELECT id, visit_date, people
FROM (
  SELECT
    g.*,
    COUNT(*) OVER (PARTITION BY grp_key) AS island_len
  FROM grp AS g
) x
WHERE island_len >= 3
ORDER BY visit_date;

Runtime 353 ms
Beats 55.35%

```

**ポイント**

* `big_islands` と `JOIN` を削除 → マテリアライズや結合コストを削減
* 同一 `grp_key`（連番の島）内の行数を `COUNT(*) OVER (PARTITION BY grp_key)` で算出し、外側で `WHERE island_len >= 3`
* 可読性も向上

実行計画上は「`ORDER BY id` のウィンドウ → `PARTITION BY grp_key` のウィンドウ → 最終フィルタ」の二段で済みます。

---

## 代替の等価書き換え（`ROW_NUMBER` を 1 回に）

MySQL は同一 SELECT 句でエイリアスを別のウィンドウ関数の `PARTITION BY` に直接使えないため、上のように 2 段に分けます。もし 1 段に詰めたい場合は、CTE を 1 個にして派生表で包むのが最小です。

```sql
SELECT id, visit_date, people
FROM (
  SELECT
    id,
    visit_date,
    people,
    COUNT(*) OVER (PARTITION BY (id - ROW_NUMBER() OVER (ORDER BY id))) AS island_len
  FROM Stadium
  WHERE people >= 100
) t
WHERE island_len >= 3
ORDER BY visit_date;

Error
0 / 15 testcases passed
You cannot nest a window function in the specification of window '<unnamed window>'.
```

> ただし上記は一部バージョンでオプティマイザが式の再計算を増やす可能性があるため、安定運用なら **CTE 2 段**（前掲の改善版）を推奨します。

---

## インデックス最適化

フィルタが `people >= 100`、ウィンドウが `ORDER BY id`、出力で `visit_date` を返すため、次を推奨します。

```sql
-- people で範囲抽出しつつ id の順序性を活かす
CREATE INDEX ix_stadium_people_id ON Stadium (people, id);

-- さらにカバリングさせたいなら（ストレージと更新コストと相談）
CREATE INDEX ix_stadium_people_id_date ON Stadium (people, id, visit_date);
```

効果：

* `pre` で `people` 条件の範囲スキャン
* そのまま `id` 昇順の並びを得やすく、`ROW_NUMBER() OVER (ORDER BY id)` のソートコストを低減
* 最終 `ORDER BY visit_date` は別ソートになりますが、対象行は **島長 ≥ 3** に絞られているためコストは小さくなります
  （要件的には `visit_date ASC` 必須ですが、仕様上「id ↑ ⇒ date ↑」なので、許容される環境なら `ORDER BY id` で等価にできます）

---

## 追加の微調整

* データ量が少ない/中程度なら現状でも十分。大規模（数百万行〜）なら統計更新と `ANALYZE TABLE Stadium;` を適宜実施。
* CTE は MySQL 8 では多くの場合インライン化されますが、環境によっては派生表のマテリアライズが起きます。実行計画を見て重い場合は **派生表に `/*+ NO_MERGE() */` / `/*+ MERGE() */` ヒント**の検討（バージョン依存）も。

---

## まとめ

* **JOIN を外し、ウィンドウ `COUNT()` で直接フィルタ**：短く速く
* **`(people, id[, visit_date])` の複合インデックス**：ソート・走査コスト削減
* これで一般に **数〜十数 % 程度の短縮**が見込めます（データ分布とバージョン次第）

MySQL 8 は **ウィンドウ関数の“入れ子”を禁止**しており、`PARTITION BY (id - ROW_NUMBER() OVER (...))` のような書き方はできません。そのため、**`ROW_NUMBER()` を先に別レイヤーで計算してから**、外側で `COUNT() OVER (PARTITION BY ...)` を使う形に分解してください。

## 動く修正版（派生表2段でネスト回避）

```sql
SELECT id, visit_date, people
FROM (
  SELECT
    t.*,
    COUNT(*) OVER (PARTITION BY (id - rn)) AS island_len
  FROM (
    SELECT
      id,
      visit_date,
      people,
      ROW_NUMBER() OVER (ORDER BY id) AS rn
    FROM Stadium
    WHERE people >= 100
  ) AS t
) AS x
WHERE island_len >= 3
ORDER BY visit_date;

Runtime 332 ms
Beats 80.65%

```

* 内側：`ROW_NUMBER()` を `rn` として計算
* 中間：`grp_key = id - rn` を式で作る（ここでは単なる通常列演算）
* 外側：`COUNT(*) OVER (PARTITION BY (id - rn))` で島の長さを算出して `>= 3` を抽出

> ポイント：**ウィンドウ関数の引数や `PARTITION BY` 式の中に別のウィンドウ関数を置かない**こと。必ず一段外に出してから使う。

## CTE 版（読みやすさ重視・推奨）

```sql
WITH pre AS (
  SELECT id, visit_date, people
  FROM Stadium
  WHERE people >= 100
),
grp AS (
  SELECT
    id,
    visit_date,
    people,
    id - ROW_NUMBER() OVER (ORDER BY id) AS grp_key
  FROM pre
)
SELECT id, visit_date, people
FROM (
  SELECT
    g.*,
    COUNT(*) OVER (PARTITION BY grp_key) AS island_len
  FROM grp AS g
) x
WHERE island_len >= 3
ORDER BY visit_date;
```

こちらは既にご提案済みの「JOIN 省略版」で、**ネストなし**・可読性良好です。

---

### 参考メモ

* MySQL 8 の制約：`You cannot nest a window function in the specification of window ...`
  → **サブクエリ（派生表 or CTE）で段階計算**が定石です。
* パフォーマンス面では、どちらの書き方も**結合を無くし、スキャン回数を減らせる**ため、先の `big_islands` 版より有利になりやすいです。
* 追加最適化：`CREATE INDEX ix_stadium_people_id ON Stadium(people, id);` は引き続き有効です。

